In [1]:
%%HTML
<h2>About this Project</h2>
<p>In this Project, you will find the gradient of the logistic loss function and implement gradient descent in order to create a linear classifier that can "filter" spam email messages.</p>

<h3>Evaluation</h3>

<p><strong>This project must be successfully completed and submitted in order to receive credit for this course. Your score on this project will be included in your final grade calculation.</strong><p>
    
<p>You are expected to write code where you see <em># YOUR CODE HERE</em> within the cells of this notebook. Not all cells will be graded; code input cells followed by cells marked with <em>#Autograder test cell</em> will be graded. Upon submitting your work, the code you write at these designated positions will be assessed using an "autograder" that will run all test cells to assess your code. You will receive feedback from the autograder that will identify any errors in your code. Use this feedback to improve your code if you need to resubmit. Be sure not to change the names of any provided functions, classes, or variables within the existing code cells, as this will interfere with the autograder. Also, remember to execute all code cells sequentially, not just those you’ve edited, to ensure your code runs properly.</p>
    
<p>You can resubmit your work as many times as necessary before the submission deadline. If you experience difficulty or have questions about this exercise, use the Q&A discussion board (found in the Live Labs section of this course) to engage with your peers or seek assistance from the instructor.<p>

<p>Before starting your work, please review <a href="https://s3.amazonaws.com/ecornell/global/eCornellPlagiarismPolicy.pdf">eCornell's policy regarding plagiarism</a> (the presentation of someone else's work as your own without source credit).</p>

<h3>Submit Code for Autograder Feedback</h3>

<p>Once you have completed your work on this notebook, you will submit your code for autograder review. Follow these steps:</p>

<ol>
    <li><strong>Save your notebook —</strong> Click <strong>Save and Checkpoint</strong> in the "File" menu.</li>
  <li><strong>Mark as Completed —</strong> In the blue menu bar along the top of this code exercise window, you’ll see a menu item called <strong>Education</strong>. In the <strong>Education</strong> menu, click <strong>Mark as Completed</strong> to submit your code for autograder/instructor review. This process will take a moment and a progress bar will show you the status of your submission.</li>
	<li><strong>Review your results —</strong> Once your work is marked as complete, the results of the autograder will automatically be presented in a new tab within the code exercise window. You can click on the assessment name in this feedback window to see more details regarding specific feedback/errors in your code submission.</li>
  <li><strong>Repeat, if necessary —</strong> The Jupyter notebook will always remain accessible in the first tabbed window of the exercise. To reattempt the work, you will first need to click <strong>Mark as Uncompleted</strong> in the <strong>Education</strong> menu and then proceed to make edits to the notebook. Once you are ready to resubmit, follow steps one through three. You can repeat this procedure as many times as necessary.</li>
</ol>

In [2]:
%%HTML
<h2>Getting Started</h2>
<h3>Python Initialization</h3> 

Please run the following code to initialize your Python kernel. You should be running a version of Python 3.x. </p>

In [ ]:
import numpy as np
import os
from pylab import *
import matplotlib.pyplot as plt
%matplotlib inline 

from helper import *

print('You\'re running python %s' % sys.version.split(' ')[0])

In [ ]:
Next, let's simulate some 2D data with binary (0/1) labels.  You'll be generating this data from non-overlapping multivariate normal distributions that should be very easily separable for logistic regression.

In [ ]:
np.random.seed(12)
n_samples = 500

class_one = np.random.multivariate_normal([5, 10], [[1, .25],[.25, 1]], n_samples)
class_one_labels = -np.ones(n_samples)

class_two = np.random.multivariate_normal([0, 5], [[1, .25],[.25, 1]], n_samples)
class_two_labels = np.ones(n_samples)

features = np.vstack((class_one, class_two))
labels = np.hstack((class_one_labels, class_two_labels))

In [ ]:
# Let's see what what our feature arrays look like. 
features

In [ ]:
# We can visualize these data distributions
plt.figure(figsize=(9, 6))
plt.scatter(features[:, 0], features[:, 1],
            c = labels, alpha = .6);

plt.title("Binary labeled data in 2D", size=15);
plt.xlabel("Feature 1", size=13);
plt.ylabel("Feature 2", size=13);

In [2]:
%%HTML
<p>In logistic regression, we use gradient ascent to solve for the weight vector that maximizes the (log) likelihood of observing the data.  (Equivalently, we can use gradient _descent_ to solve for the weight vector that _minimizes_ the _negative_ log likelihood - refer to Module 3 if you need a review of this derivation!)</p>

In [1]:
%%HTML
<h2>Gradient Descent and Email Spam Classification</h2>

<h3>Part One: Sigmoid [Graded]</h3>

<p>To begin, you must first implement the sigmoid function:    $\sigma(z)=\frac{1}{1+e^{-z}}$ </p>


In [ ]:
def sigmoid(z):
    # Input: 
    # z : scalar or array of dimension n 
    # Output:
    # sgmd: scalar or array of dimension n
    
    # YOUR CODE HERE
    
    
    sgmd = 1/(1+np.exp(-z))
    sgmd
    return sgmd
    raise NotImplementedError()
    
# h = np.random.rand(10) # input is an 10-dimensional array
# sgmd1 = sigmoid(h)

In [ ]:
def test_sigmoid1():
    h = np.random.rand(10) # input is an 10-dimensional array
    sgmd1 = sigmoid(h)
    return sgmd1.shape == h.shape # output should be a 10-dimensional array

def test_sigmoid2():
    h = np.random.rand(10) # input is an 10-dimensional array
    sgmd1 = sigmoid(h) # compute the sigmoids with your function
    sgmd2 = sigmoid_grader(h) # compute the sigmoids with ground truth funcdtion
    return (np.linalg.norm(sgmd1 - sgmd2) < 1e-5) # check if they agree

def test_sigmoid3():
    x = np.random.rand(1) # input is a scalar
    sgmd1 = sigmoid(x) # compute the sigmoids with your function
    sgmd2 = sigmoid_grader(x) # compute the sigmoids with ground truth function
    return (np.linalg.norm(sgmd1 - sgmd2) < 1e-5) # check if they agree

def test_sigmoid4():
    x = np.array([-1e10,1e10,0]) # three input points: very negative, very positive, and 0
    sgmds = sigmoid(x) # compute the sigmoids with your function
    truth = np.array([0,1,0.5]) # the truth should be 0, 1, 0.5 exactly
    return (np.linalg.norm(sgmds - truth) < 1e-8) # test if this is true


runtest(test_sigmoid1, 'test_sigmoid1')
runtest(test_sigmoid2, 'test_sigmoid2')
runtest(test_sigmoid3, 'test_sigmoid3')
runtest(test_sigmoid4, 'test_sigmoid4')

In [3]:
%%HTML
<h3>Part Two: Implement <code>y_pred</code> [Graded]</h3>

<p>We consider binary logistic regression with class labels $y\in\{+1,-1\}$. Implement a function <code>y_pred(X,w)</code> that computes $P(y=1\;|\;\mathbf{x};\mathbf{w}, b)$ for each row-vector $\mathbf{x}$ in the matrix <code>X</code>.</p>
<p>Recall that:
$$P(y\;|\;\mathbf{x};\mathbf{w})=\sigma(y (\mathbf{w}^\top \mathbf{x} + b))$$
</p>
<br />

In [ ]:
def y_pred(X, w, b=0):
    # Input:
    # X: nxd matrix
    # w: d-dimensional vector
    # b: scalar (optional, if not passed on is treated as 0)
    # Output:
    # prob: n-dimensional vector
    
    # YOUR CODE HERE
    
    prob = sigmoid(np.inner(X,w.T) + b)
    
    
    #print('X: ', X.shape)
    #print('W: ', w.shape)
    #print('prob: ', prob.shape)
    return prob
    raise NotImplementedError()
    
n = 20
d = 5
X = np.random.rand(n,d) # generate n random vectors with d dimensions
w = np.random.rand(5) # define a random weight vector
probs=y_pred(X,w,0) 

In [ ]:
def test_ypred1():
    n = 20
    d = 5
    X = np.random.rand(n,d) # generate n random vectors with d dimensions
    w = np.random.rand(5) # define a random weight vector
    probs=y_pred(X,w,0) # compute the probabilities of P(y=1|x;w) using your y_pred function
    return probs.shape == (n, ) # check if all outputs are >=0 and <=1


def test_ypred2():
    n = 20
    d = 5
    X = np.random.rand(n,d) # generate n random vectors with d dimensions
    w = np.random.rand(5) # define a random weight vector
    probs=y_pred(X, w, 0) # compute the probabilities of P(y=1|x;w) using your y_pred function
    return all(probs>=0) and all(probs<=1) # check if all outputs are >=0 and <=1

def test_ypred3():
    n = 20
    d = 5
    X = np.random.rand(n,d) # generate n random vectors with d dimensions
    w = np.random.rand(5) # define a random weight vector
    probs1=y_pred(X, w, 0) # compute the probabilities of P(y=1|x;w) using your y_pred function
    probs2=y_pred(X,-w, 0) # compute the probabilities of P(y=1|x;w) using your y_pred function
    return np.linalg.norm(probs1+probs2-1)<1e-08 # check if P(y|x;w)+P(y|x;-w)=1



def test_ypred4():
    X=np.random.rand(25,4) # define random input
    w=np.array([1,0,0,0]) # all-zeros weight vector
    prob=y_pred(X, w, 0) # compute P(y|X;w)
    truth=sigmoid(X[:,0]) # should simply be the sigmoid of the first feature
    return np.linalg.norm(prob-truth)<1e-08 # see if they match


def test_ypred5(): 
    X=np.array([[0.61793598, 0.09367891], # define 3 inputs (2D)
               [0.79447745, 0.98605996],
               [0.53679997, 0.4253885 ]])
    w=np.array([0.9822789 , 0.16017851]); # define weight vector
    prob=y_pred(X, w, 3) # compute P(y|X;w)
    truth=np.array([0.97396645,0.98089179,0.97328431]) # this is the grount truth
    return np.linalg.norm(prob-truth)<1e-08 # see if they match

runtest(test_ypred1, 'test_ypred1')
runtest(test_ypred2, 'test_ypred2')
runtest(test_ypred3, 'test_ypred3')
runtest(test_ypred4, 'test_ypred4')
runtest(test_ypred5, 'test_ypred5')

In [4]:
%%HTML
<h3>Part Three: Implement <code>log_loss</code> [Graded]</h3>

<p>Assume you are given a data set $\mathbf{x}_1,\dots,\mathbf{x}_n$ stored as row-vectors in a matrix <code>X</code> with labels $y_1,\dots,y_n$ stored in vector <code>y</code>. Compute the <b>negative</b> log-likelihood (<code>log_loss</code>) of all inputs
    $$NLL=-\log P(\mathbf{y}\;|\;\mathbf{X};\mathbf{w}, b)=-\sum_{i=1}^n \log\left((P(y_i\;|\; \mathbf{x}_i;\mathbf{w}, b))\right)=-\sum_{i=1}^n \log\left(\sigma(y_i(\mathbf{w}^\top \mathbf{x}_i + b))\right).$$
    </p>

In [ ]:
def log_loss(X, y, w, b=0):
    # Input:
    # X: nxd matrix
    # y: n-dimensional vector with labels (+1 or -1)
    # w: d-dimensional vector
    # Output:
    # a scalar
    assert np.sum(np.abs(y))==len(y) # check if all labels in y are either +1 or -1
    
    # YOUR CODE HERE
    #W = (X.T.dot(y))/(X.T.dot(X))
    #print('W: ', W.shape)
    #print('y', y.shape)
    
   # nll = -np.sum(np.log(1 + np.exp(-y*np.inner(w.T,X)))
   # nll = -np.sum(np.log(1-y_pred(X,w,b)))
    #score = np.dot(X,w)
    #nll = -np.sum(y*scores - np.log(1+ np.exp(scores)))
    #nll = -np.log(y_pred(X,w,b))
    #nll = (-y * np.log(y_pred(X,w,b)) - (1 - y) * np.log(1 - y_pred(X,w,b))).mean() 
    #z = np.dot(X, w)
    #nll = -np.sum( y*z - np.log(1 + np.exp(z)) )
    
    #nll = -np.sum(np.log(y_pred(X,w,b)))
    #nll = np.sum(-y*np.log(y_pred(X,w,b)) - (1 - y)*np.log(1 - y_pred(X,w,b)))
    #nll = -np.sum(y@np.log(y_pred(X,w,b)))

    #nll = -np.sum(np.log(y_pred(X,w,b)))

    nll3 = -np.sum(np.log(sigmoid(y*(np.inner(w.T,X) +b))))
    #nll2 = -np.sum(np.log(sigmoid(y@np.inner(w.T,X)+b)))
    
    #print('y: ', y.shape)
    #print('Partial: ', nll3)     
    #print('nll2: ', nll2)              
    #print('nll: ', nll)
    return nll3
    raise NotImplementedError()

np.random.seed(0)
X = np.random.rand(500,15) # generate n random vectors with d dimensions
w = np.random.rand(15) # define a random weight vector
b = np.random.rand(1) # define a bias
y = (np.random.rand(500)>0.5)*2-1;
ll=log_loss(X,y,w, b) #
ll2 = log_loss_grader(X,y,w,b)
print('ll2: ', ll2)

In [ ]:
def test_logloss1():
    X = np.random.rand(25,5) # generate n random vectors with d dimensions
    w = np.random.rand(5) # define a random weight vector
    b = np.random.rand(1) # define a bias
    y = np.ones(25) # set labels all-(+1)
    ll=log_loss(X,y,w, b) # compute the probabilities of P(y=1|x;w) using your y_pred function
    # if labels are all-ones function becomes simply the sum of log of y_pred
    return np.isscalar(ll) # check whether the output is a scalar

def test_logloss2():
    X = np.random.rand(25,5) # generate n random vectors with d dimensions
    w = np.random.rand(5) # define a random weight vector
    b = np.random.rand(1) # define a bias
    y = np.ones(25) # set labels all-(+1)
    ll=log_loss(X,y,w, b) # compute the probabilities of P(y=1|x;w) using your y_pred function
    ll2=-np.sum(np.log(y_pred(X, w, b))) # if labels are all-ones function becomes simply the sum of log of y_pred
    return np.linalg.norm(ll-ll2)<1e-05

def test_logloss3():
    X = np.random.rand(25,5) # generate n random vectors with d dimensions
    w = np.random.rand(5) # define a random weight vector
    b = np.random.rand(1) # define a bias
    y = -np.ones(25) # set labels all-(-1)
    ll=log_loss(X,y,w,b) # compute the probabilities of P(y=1|x;w) using your y_pred function
    ll2=-np.sum(np.log(1-y_pred(X,w, b))) # if labels are all-ones function becomes simply the sum of log of 1-y_pred
    return np.linalg.norm(ll-ll2)<1e-05

def test_logloss4():
    X = np.random.rand(20,5) # generate n random vectors with d dimensions
    w = np.array([0,0,0,0,0]) # define an all-zeros weight vector
    y = (np.random.rand(20)>0.5)*2-1; # define n random labels (+1 or -1)
    ll=log_loss(X,y,w,0) # compute the probabilities of P(y=1|x;w) using your y_pred function
    # the log-likelihood for each of the 20 examples should be exactly 0.5:
    return np.linalg.norm(ll+20*np.log(0.5))<1e-08 

def test_logloss5():
    X = np.random.rand(500,15) # generate n random vectors with d dimensions
    w = np.random.rand(15) # define a random weight vector
    b = np.random.rand(1) # define a bias
    y = (np.random.rand(500)>0.5)*2-1; # define n random labels (+1 or -1)
    ll=log_loss(X,y,w,b) # compute the probabilities of P(y=1|x;w) using your y_pred function
    ll2=log_loss_grader(X,y,w,b) # compute the probabilities of P(y=1|x;w) using your y_pred function
    return np.linalg.norm(ll-ll2)<1e-05

runtest(test_logloss1, 'test_logloss1')
runtest(test_logloss2, 'test_logloss2')
runtest(test_logloss3, 'test_logloss3')
runtest(test_logloss4, 'test_logloss4')
runtest(test_logloss5, 'test_logloss5')

In [5]:
%%HTML
<h3>Part Four: Compute Gradient [Graded]</h3>

Now, verify that the gradient of the log-loss with respect to the weight vector is:
$$\frac{\partial NLL(\mathbf{X},\mathbf{y},\mathbf{w}, b)}{\partial \mathbf{w}}=\sum_{i=1}^n -y_i\sigma(-y_i(\mathbf{w}^\top\mathbf{x}_i + b))\mathbf{x}_i.$$ 

$$\frac{\partial NLL(\mathbf{X},\mathbf{y},\mathbf{w}, b)}{\partial b}=\sum_{i=1}^n -y_i\sigma(-y_i(\mathbf{w}^\top\mathbf{x}_i + b)).$$
Implement the function <code>gradient</code> which returns the first derivative with respect to <code>w, b</code> for a given <code>X, y, w, b</code>.

Hint: remember that you derived earlier that $\sigma'(z)=\sigma(z)(1-\sigma(z))$.

In [ ]:
def gradient(X, y, w, b):
    # Input:
    # X: nxd matrix
    # y: n-dimensional vector with labels (+1 or -1)
    # w: d-dimensional vector
    # b: a scalar bias term
    # Output:
    # wgrad: d-dimensional vector with gradient
    # bgrad: a scalar with gradient
    
    n, d = X.shape
    wgrad = np.zeros(d)
    bgrad = 0.0
    # YOUR CODE HERE
    
    
    h = y - y_pred(X,w,b)
    #wgrad = np.sum*y-y_pred(X,w,b)*X
    
    #wgrad = np.dot(X.T,h)
    wgrad = -y*(sigmoid(-y*(np.inner(w.T,X) +b)))@X

    partialx = -y*(sigmoid(-y*(np.inner(w.T,X) +b)))@X
    bgrad = np.sum(-y*(sigmoid(-y*(np.inner(w.T,X) +b))))


    
    return wgrad, bgrad
 
# X = np.random.rand(25,5) # generate n random vectors with d dimensions
# w = np.random.rand(5) # define a random weight vector
# b = np.random.rand(1) # define a bias
# y = (np.random.rand(25)>0.5)*2-1 # set labels all-(+1)
# wgrad, bgrad = gradient(X, y, w, b)

In [ ]:
def test_grad1():
    X = np.random.rand(25,5) # generate n random vectors with d dimensions
    w = np.random.rand(5) # define a random weight vector
    b = np.random.rand(1) # define a bias
    y = (np.random.rand(25)>0.5)*2-1 # set labels all-(+1)
    wgrad, bgrad = gradient(X, y, w, b) # compute the gradient using your function
    
    return wgrad.shape == w.shape and np.isscalar(bgrad)


def test_grad2():
    X = np.random.rand(25,5) # generate n random vectors with d dimensions
    w = np.random.rand(5) # define a random weight vector
    b = np.random.rand(1) # define a bias
    y = (np.random.rand(25)>0.5)*2-1 # set labels all-(+1)
    wgrad, bgrad = gradient(X, y, w, b) # compute the gradient using your function
    wgrad2, bgrad2 = gradient_grader(X, y, w, b) # compute the gradient using ground truth
    return np.linalg.norm(wgrad - wgrad2)<1e-06 and np.linalg.norm(bgrad - bgrad2) < 1e-06 # test if they match

def test_grad3():
    X = np.random.rand(25,5) # generate n random vectors with d dimensions
    y = (np.random.rand(25)>0.5)*2-1 # set labels all-(+1)
    w = np.random.rand(5) # define a random weight vector
    b = np.random.rand(1) 

    w_s = np.random.rand(5)*1e-05 # define tiny random step 
    b_s = np.random.rand(1)*1e-05 # define tiny random step 
    ll1 = log_loss(X,y,w+w_s, b+b_s)  # compute log-likelihood after taking a step
    
    ll = log_loss(X,y,w,b) # use Taylor's expansion to approximate new loss with gradient
    wgrad, bgrad =gradient(X,y,w,b) # compute gradient
    ll2=ll+ wgrad @ w_s + bgrad * b_s # take linear step with Taylor's approximation
    return np.linalg.norm(ll1-ll2)<1e-05 # test if they match

def test_grad4():
    w1, b1, losses1 = logistic_regression_grader(features, labels, 1000, 1e-03, gradient)
    w2, b2, losses2 = logistic_regression_grader(features, labels, 1000, 1e-03)
    return(np.abs(losses1[-1]-losses2[-1])<0.1)

runtest(test_grad1, 'test_grad1')
runtest(test_grad2, 'test_grad2')
runtest(test_grad3, 'test_grad3')
runtest(test_grad4, 'test_grad4')

In [6]:
%%HTML
<h3>Part Five: Weight Update of Gradient Ascent [Graded]</h3>
    
Write code below to implement the weight update of gradient descent on the log_loss function.  Hint: use the `gradient` and `log_loss` functions from above. Please use a <b>constant</b> learning rate throughout (i.e. do not decrease the learning rate).

In [ ]:
def logistic_regression(X, y, max_iter, alpha):
    n, d = X.shape
    w = np.zeros(d)
    b = 0.0
    #losses = np.zeros(max_iter)    
    losses = []
    for step in range(max_iter):
        # YOUR CODE HERE
        
        #Get wgradient and b gradient
        
        wgrad, bgrad = gradient(X,y, w,b)
        
        #update w
        w = w - alpha*wgrad
        
        #update b
        b = b -alpha*bgrad
        
        #define losses
        losses.append(log_loss(X,y,w,b))
        
    
    return w, b, losses

weight, b, losses = logistic_regression(features, labels, 1000, 1e-04)
plot(losses)
xlabel('iterations')
ylabel('log_loss')
# your loss should go down :-)

In [ ]:
def test_logistic_regression1():

    XUnit = np.array([[-1,1],[-1,0],[0,-1],[-1,2],[1,-2],[1,-1],[1,0],[0,1],[1,-2],[-1,2]])
    YUnit = np.hstack((np.ones(5), -np.ones(5)))

    w1, b1, _ = logistic_regression(XUnit, YUnit, 30000, 5e-5)
    w2, b2, _ = logistic_regression_grader(XUnit, YUnit, 30000, 5e-5)
    return (np.linalg.norm(w1 - w2) < 1e-5) and (np.linalg.norm(b1 - b2) < 1e-5)

def test_logistic_regression2():
    X = np.vstack((np.random.randn(50, 5), np.random.randn(50, 5) + 2))
    Y = np.hstack((np.ones(50), -np.ones(50)))
    max_iter = 300
    alpha = 1e-5
    w1, b1, _ = logistic_regression(X, Y, max_iter, alpha)
    w2, b2, _ = logistic_regression_grader(X, Y, max_iter, alpha)
    return (np.linalg.norm(w1 - w2) < 1e-5) and (np.linalg.norm(b1 - b2) < 1e-5)

def test_logistic_regression3(): # check if losses match predictions
    X = np.vstack((np.random.randn(50, 5), np.random.randn(50, 5) + 2))
    Y = np.hstack((np.ones(50), -np.ones(50)))
    max_iter = 30
    alpha = 1e-5
    w1, b1, losses1 = logistic_regression(X, Y, max_iter, alpha)
    return np.abs(log_loss(X,Y,w1,b1)-losses1[-1])<1e-09

def test_logistic_regression4(): # check if loss decreases
    X = np.vstack((np.random.randn(50, 5), np.random.randn(50, 5) + 2))
    Y = np.hstack((np.ones(50), -np.ones(50)))
    max_iter = 30
    alpha = 1e-5
    w1, b1, losses1 = logistic_regression(X, Y, max_iter, alpha)
    return losses[-1]<losses[0]

runtest(test_logistic_regression1, 'test_logistic_regression1')
runtest(test_logistic_regression2, 'test_logistic_regression2')
runtest(test_logistic_regression3, 'test_logistic_regression3')
runtest(test_logistic_regression4, 'test_logistic_regression4')

In [3]:
%%HTML
<h2>Original Data</h2>
<p>Now run your implementation on the binary classification data from the top of the notebook.  Check your code by plotting the values of the negative log likelihood - should these values increase or decrease as the number of iterations grows?  Do your values move in the right direction?</p>

You can tune `max_iter` and `alpha` to see how they affect convergence!  

In [ ]:
max_iter = 10000
alpha = 1e-4
final_w, final_b, losses = logistic_regression(features, labels, max_iter, alpha)

plt.figure(figsize=(9, 6))
plt.plot(losses)
plt.title("Loss vs. iteration", size=15)
plt.xlabel("Num iteration", size=13)
plt.ylabel("Loss value", size=13)

In [4]:
%%HTML
<p>Below, we will take the final weights from the logistic solver and predict labels for the entire dataset.  By plotting the results, we can get a sense of where the linear decision boundary lies.  What do you notice?  What could be changed to further improve the accuracy of the classifier? (_Hint: take a look at the second video in Module 1._)</p>

In [ ]:
scores = y_pred(features, final_w, final_b)

pred_labels = (scores > 0.5).astype(int)
pred_labels[pred_labels != 1] = -1

plt.figure(figsize=(9, 6))

# plot the decision boundary 
x = np.linspace(np.amin(features[:, 0]), np.amax(features[:, 0]), 10)
y = -(final_w[0] * x + final_b)/ final_w[1] 
plt.plot(x, y)

plt.scatter(features[:, 0], features[:, 1],
            c = pred_labels, alpha = .6)
plt.title("Predicted labels", size=15)
plt.xlabel("Feature 1", size=13)
plt.ylabel("Feature 2", size=13)
plt.axis([-3,10,0,15])

In [7]:
%%HTML
<h1>Build an Email Spam Filter</h1>
<p> With logistic regression implemented, you can now build an email spam filter using logistic regression. The functions below load in pre-processed email data from cloud storage, where emails are represented as one-hot vectors.</p>

In [ ]:
import pandas as pd
import dask
import dask.bag
from dask.diagnostics import ProgressBar


train_url = 's3://codio/CIS530/CIS533/data_train'
test_url = 's3://codio/CIS530/CIS533/data_test'

In [ ]:
# tokenize the email and hashes the symbols into a vector
def extract_features_naive(email, B):
    # initialize all-zeros feature vector
    v = np.zeros(B)
    email = ' '.join(email)
    # breaks for non-ascii characters
    tokens = email.split()
    for token in tokens:
        v[hash(token) % B] = 1
    return v

def load_spam_data(extract_features, B=512, url=train_url):
    '''
    INPUT:
    extractfeatures : function to extract features
    B               : dimensionality of feature space
    path            : the path of folder to be processed
    
    OUTPUT:
    X, Y
    '''
    
    all_emails = pd.read_csv(url+'/index', header=None).values.flatten()
    
    xs = np.zeros((len(all_emails), B))
    ys = np.zeros(len(all_emails))
    
    labels = [k.split()[0] for k in all_emails]
    paths = [url+'/'+k.split()[1] for k in all_emails]

    ProgressBar().register()
    dask.config.set(scheduler='threads', num_workers=50)
    bag = dask.bag.read_text(paths, storage_options={'anon':True})
    contents = dask.bag.compute(*bag.to_delayed())
    for i, email in enumerate(contents):
        # make labels +1 for "spam" and -1 for "ham"
        ys[i] = (labels[i] == 'spam') * 2 - 1
        xs[i, :] = extract_features(email, B)
    print('Loaded %d input emails.' % len(ys))
    return xs, ys

Xspam, Yspam = load_spam_data(extract_features_naive)
Xspam.shape

In [8]:
%%HTML
<h3>Split The Dataset</h3>

<p>Now that you have loaded the dataset, it's time to split it into training and testing. To evaluate your algorithm, run the code below to split off 20% of this data into a testing set, leaving 80% as your training set.</p>

In [ ]:
# Split data into training (xTr and yTr) 
# and testing (xTv and yTv)
n, d = Xspam.shape
# Allocate 80% of the data for training and 20% for testing
cutoff = int(np.ceil(0.8 * n))
# indices of training samples
xTr = Xspam[:cutoff,:]
yTr = Yspam[:cutoff]
# indices of testing samples
xTv = Xspam[cutoff:]
yTv = Yspam[cutoff:]

In [9]:
%%HTML
<h3>Training and Evaluating </h3>

<p> Running the following cell will produce a logistic regression model that can classify unseen emails at roughly 88% accuracy</p>

In [ ]:
max_iter = 5000
alpha = 1e-5
final_w_spam, final_b_spam, losses = logistic_regression(xTr, yTr, max_iter, alpha)

plt.figure(figsize=(9, 6))
plt.plot(losses)
plt.title("Loss vs. iteration", size=15)
plt.xlabel("Num iteration", size=13)
plt.ylabel("Loss value", size=13)

# evaluate training accuracy
scoresTr = y_pred(xTr, final_w_spam, final_b_spam)
pred_labels = (scoresTr > 0.5).astype(int)
pred_labels[pred_labels != 1] = -1
trainingacc = np.mean(pred_labels == yTr)

# evaluate testing accuracy
scoresTv = y_pred(xTv, final_w_spam, final_b_spam)
pred_labels = (scoresTv > 0.5).astype(int)
pred_labels[pred_labels != 1] = -1
validationacc = np.mean(pred_labels==yTv)
print("Training accuracy %2.2f%%\nValidation accuracy %2.2f%%\n" % (trainingacc*100,validationacc*100))

In [10]:
%%HTML
<h2>Challenge: Improve Your Spam Classifier <b>[Ungraded]</b></h2>

<p>You can improve your classifier in two ways:</p>

<ol>
<li><b>Feature Extraction</b>:
Modify the function <code>extract_features_comp()</code>.
This function takes in a file path <code>path</code> and
a feature dimension <code>B</code> and should output a feature vector of dimension <code>B</code>.
The autograder will pass in a file path pointing to a file that contains an email,
and set <code>B</code> = <code>feature_dimension</code>.
We provide a naive feature extraction as an example.
</li>
<li><b>Model Training</b>:
Modify the function <code>train_spam_filter_comp()</code>.
This function takes in training data <code>xTr</code> and training labels <code>yTr</code> and
should output a weight vector <code>w</code> and bias <code>b</code> for linear classification. The predictions will be calculated exactly the same way as we have demonstrated in the previous cell. 
We provide an initial implementation using gradient descent and logistic regression.
</li>
</ol>

<p>Your model will be trained on the same training set above (loaded by <code>load_spam_data()</code>), but we will test its accuracy on a secret dataset of emails.</p>

In [ ]:
feature_dimension = 512
def extract_features_challenge(email, B=feature_dimension):
    '''
    INPUT:
    path : file path of email
    B    : dimensionality of feature vector
    
    OUTPUTS:
    x    : B dimensional vector
    '''
    # initialize all-zeros feature vector
    v = np.zeros(B)
    email = ' '.join(email)
    # breaks for non-ascii characters
    tokens = email.split()
    for token in tokens:
        v[hash(token) % B] = 1
        
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return v

In [ ]:
def train_spam_filter_challenge(xTr, yTr):
    '''
    INPUT:
    xTr : nxd dimensional matrix (each row is an input vector)
    yTr : d   dimensional vector (each entry is a label)
    
    OUTPUTS:
    w : d dimensional vector for linear classification
    '''
    n, d = xTr.shape
    
    max_iter = 100
    alpha = 1e-5
    w, b, losses = logistic_regression(xTr, yTr, max_iter, alpha)
    
    # YOUR CODE HERE
    raise NotImplementedError()

    return w, b

In [ ]:
def challenge_selftest():
    xTr,yTr = load_spam_data(extract_features_challenge, feature_dimension, train_url)
    w, b = train_spam_filter_challenge(xTr, yTr)
    xTe,yTe = load_spam_data(extract_features_challenge, feature_dimension, test_url)
    scoresTe = sigmoid_grader(xTe @ w + b)
    
    preds = (scoresTe > 0.5).astype(int)
    preds[preds != 1] = -1
    
    pos_ind = (yTe == 1)
    neg_ind = (yTe == -1)
    
    pos_acc = np.mean(yTe[pos_ind] == preds[pos_ind])
    neg_acc = np.mean(yTe[neg_ind] == preds[neg_ind])
    
    test_accuracy = 0.5*pos_acc + 0.5*neg_acc
    
    scoresTr =  sigmoid_grader(xTr @ w + b)
    preds_Tr = (scoresTr > 0.5).astype(int)
    preds_Tr[preds_Tr != 1] = -1
    
    training_accuracy = np.mean(preds_Tr == yTr)
    return training_accuracy, test_accuracy

training_acc, test_acc = challenge_selftest()
print("Your features and model achieved training accuracy: {:.2f}% and test accuracy: {:.2f}%".format(training_acc*100, test_acc*100))